Let's first try if a Neural Network is able to obtain better results than our causal model.

### Imports and stuff

In [1]:
from PC_deletable import PC
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

In [2]:
data = pd.read_csv('../data/train.csv')
data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], inplace=True)

data['Sex'] = data['Sex'].apply(lambda x: 1 if x=='male' else 0)

data.dropna(inplace=True)

data_train = data.sample(frac=0.8, random_state=42)
data_test = data.drop(data_train.index)

### Basic Neural Network

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import f1_score

# Define the input shape
input_shape = (len(data.columns) - 1,)

# Create the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=input_shape))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare the input and target data
X = data_train.drop(columns=['Survived']).values
y = data_train['Survived'].values

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Evaluate F1 score of the model
X_test = data_test.drop(columns=['Survived']).values
y_test = data_test['Survived'].values
y_pred = model.predict(X_test) > 0.5
f1_score(y_test, y_pred)

c:\Users\jmate\miniconda3\envs\causal-inference\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6207 - loss: 0.8132
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6407 - loss: 0.7428 
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6644 - loss: 0.6730 
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7026 - loss: 0.7251 
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6945 - loss: 0.6012 
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6896 - loss: 0.6268 
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6628 - loss: 0.6108 
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6929 - loss: 0.7013 
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7062 - loss: 0.5559 
Epoch 10/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6998 - loss: 0.6432 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


0.5684210526315789

### Extract the causal graph

In [4]:
def getEdges(dictGraph):
    edges = []
    for node1, addjacents in dictGraph.items():
        for node2 in addjacents:
            edges.append((node1, node2))
    return edges

originalGraph = {column: set(data.columns.drop(column)) \
                            for column in data.columns}

G = nx.DiGraph(getEdges(originalGraph))


def discretize(data, column, bins):
    data[column] = pd.cut(data[column], bins, labels=False)

# Age and Fare discretization
discretize(data, 'Age', 5)
discretize(data, 'Fare', 5)

pc = PC(0.25, exogeneous=['Age', 'Sex'], endogeneous=['Survived'], directional=True, maxSeparatingDepth=2)

graph, separatingSets = pc.causalDiscovery(data_train)

Depth 0 completed
Depth 1 completed


### Train the GNN with the causal graph

In [5]:
import tensorflow as tf
from tf_geometric.layers import GraphConvolution
from tf_geometric.utils import normalized_adjacency

# Define the Graph Neural Network model
class GNN(tf.keras.Model):
    def __init__(self, num_features, num_classes):
        super(GNN, self).__init__()
        self.conv1 = GraphConvolution(64, activation='relu')
        self.conv2 = GraphConvolution(64, activation='relu')
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, adjacency):
        x = self.conv1(inputs, adjacency)
        x = self.conv2(x, adjacency)
        x = tf.reduce_mean(x, axis=1)
        x = self.fc(x)
        return x

# Create the GNN model
gnn = GNN(num_features=input_shape[0], num_classes=2)

# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Convert the data to TensorFlow tensors
X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.int32)
adjacency = normalized_adjacency(graph)

# Train the GNN model
for epoch in range(10):
    with tf.GradientTape() as tape:
        logits = gnn(X, adjacency)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, gnn.trainable_variables)
    optimizer.apply_gradients(zip(grads, gnn.trainable_variables))

# Evaluate the GNN model
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)
logits_test = gnn(X_test, adjacency)
predictions_test = tf.argmax(logits_test, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions_test, y_test), dtype=tf.float32))
accuracy.numpy()

ImportError: cannot import name 'GraphConvolution' from 'tf_geometric.layers' (c:\Users\jmate\miniconda3\envs\causal-inference\Lib\site-packages\tf_geometric\layers\__init__.py)